In [2]:
from llama_index.llms.gemini import Gemini
from dotenv import load_dotenv
load_dotenv()
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings

Settings.llm = Gemini(model="models/gemini-1.5-flash-latest", temperature=0)

def check_hotel_reservation_availability(date:str) -> str:
    """Check if hotel reservations are available for the given date"""
    if date == "Tomorrow" or date == "tomorrow":
        return "Yes, Reservations are available for Tomorrow"
    else:
        return "Sorry, Reservations are only available for Tomorrow at the moment"

check_hotel_reservation_availability_tool = FunctionTool.from_defaults(fn=check_hotel_reservation_availability)

In [3]:
from llama_index.embeddings.gemini import GeminiEmbedding

gemini_embedding_model = GeminiEmbedding(model_name="models/embedding-001")
Settings.embed_model = gemini_embedding_model

In [6]:
from llama_index.core import VectorStoreIndex, Settings
from llama_parse import LlamaParse
import nest_asyncio
nest_asyncio.apply()
documents = LlamaParse(result_type="markdown").load_data("./data/knowledge.pdf")

Started parsing the file under job_id f792e11e-bb3c-4620-b29f-6bc9fd104c27


In [8]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
query_engine = index.as_query_engine()

d:\learning\AI\demo-doubts\agents\agent\my_env\Lib\site-packages\tiktoken\core.py:50: RuntimeWarning: coroutine 'LlamaParse.aload_data' was never awaited
  self._core_bpe = _tiktoken.CoreBPE(mergeable_ranks, special_tokens, pat_str)


In [9]:
from llama_index.core.tools import QueryEngineTool
hotel_policy_and_services_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="Hotel",
    description="Get the policy and services related information for a hotel",
)

In [10]:
agent = ReActAgent.from_tools([check_hotel_reservation_availability_tool, hotel_policy_and_services_tool], llm=Settings.llm, verbose=True)

In [12]:
response = agent.chat(
    "I have to book a hotel reservation for tommorow please let me know if it is available and also provide me with the pricing details",
)

print(response)

> Running step 21a9e663-a690-4db5-995c-4df2db29362d. Step input: I have to book a hotel reservation for tommorow please let me know if it is available and also provide me with the pricing details
Thought: I need to check if the hotel has reservations available for tomorrow.
Action: check_hotel_reservation_availability
Action Input: {'date': 'tomorrow'}
Observation: Yes, Reservations are available for Tomorrow
> Running step fa488088-a62b-4019-a7aa-9b57c451e51b. Step input: None
Thought: I need to get the pricing details for the hotel.
Action: Hotel
Action Input: {'input': 'pricing details'}
Observation: The hotel offers three room types: Standard, Deluxe, and Executive Suite. The Standard Room costs 7,000 per night, the Deluxe Room costs 10,500 per night, and the Executive Suite costs 17,500 per night. 

> Running step 6f70f83e-08a4-4867-90f2-d668a902a172. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Yes, reservatio